In [1]:
import pandas as pd
import requests
import urllib.request 
from PIL import Image 
from io import BytesIO
from bs4 import BeautifulSoup
import os

In [2]:
# read file with links
gc_file = pd.read_excel('data/Wk33-34 Big Brand Sale Brief V1.xlsx', sheet_name = 'RD TEST')
gc_file.columns.values[0] = 'folder_name'
gc_file.head(5)

,folder_name,Image Location
0,FOOD9,https://www.petbarn.com.au/royal-canin-labrado...
1,FOOD9,https://www.petbarn.com.au/advance-delicate-tu...
2,FOOD9,https://www.petbarn.com.au/science-diet-feline...
3,FOOD9,https://www.petbarn.com.au/black-hawk-grain-fr...
4,LITTER5,https://www.petbarn.com.au/freshco-clay-clumpi...


In [ ]:
# read file with links
gc_file = pd.read_excel('data/Wk33-34 Big Brand Sale Brief V1.xlsx', sheet_name = 'RD TEST')
gc_file.columns.values[0] = 'folder_name'
gc_file.head(5)

,folder_name,Image Location
0,FOOD9,https://www.petbarn.com.au/royal-canin-labrado...
1,FOOD9,https://www.petbarn.com.au/advance-delicate-tu...
2,FOOD9,https://www.petbarn.com.au/science-diet-feline...
3,FOOD9,https://www.petbarn.com.au/black-hawk-grain-fr...
4,LITTER5,https://www.petbarn.com.au/freshco-clay-clumpi...


In [12]:
# trade promo
trade_promo = pd.read_excel('data/Trade Promo Brief Template - WK2 Catch it quick.xlsx', sheet_name= 'EDM Modules', skiprows=6)
# dropping first col since its empty
trade_promo = trade_promo.drop(trade_promo.columns[0], axis= 1)
# remove whitespace from column names
trade_promo.columns = trade_promo.columns.str.replace(' ', '')
# drop rows where module size is null - indicating modul name is empty
trade_promo = trade_promo[trade_promo['ModuleSize'].notnull()]
# retain imp cols
retain_cols = [i for i in trade_promo.columns if 'priority' in i.lower() or 'imagelink' in i.lower()]
trade_promo = trade_promo.loc[:,retain_cols]
trade_promo.columns.values[0] = 'module'
trade_promo_pivot = trade_promo.melt(id_vars= 'module', var_name= 'Image', value_name='URL')
trade_promo_pivot = trade_promo_pivot[trade_promo_pivot['URL'].str.startswith('h') & trade_promo_pivot['URL'].notnull()]
trade_promo_pivot.head(100) 

,module,Image,URL
0,FOOD1,ImageLink1,https://www.petbarn.com.au/savourlife-ancient-...
1,FOOD2,ImageLink1,https://www.petbarn.com.au/advance-total-wellb...
2,FOOD3,ImageLink1,https://www.petbarn.com.au/probalance-total-he...
3,FOOD4,ImageLink1,https://www.petbarn.com.au/pure-life-natural-b...
4,FOOD5,ImageLink1,https://www.petbarn.com.au/black-hawk-original...
...,...,...,...
121,ACC1,ImageLink3,https://www.petbarn.com.au/snooza-supa-luxe-lo...
123,ACC3,ImageLink3,https://www.petbarn.com.au/kong-dog-extreme-to...
124,ACC4,ImageLink3,https://www.petbarn.com.au/ezydog-chest-plate-...
125,ACC5,ImageLink3,https://www.petbarn.com.au/ezydog-2in1-ortho-c...


In [13]:
# Function to download an image
def download_image(url, folder_name, image_counters):
    headers = {
        'Cache-Control': 'no-cache',
        'Pragma': 'no-cache'
    }
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    img_tag = soup.find('img', {'alt':'main product photo'})
    try:
        img_url = img_tag['src']
        img_response = requests.get(img_url)
        if img_response.status_code == 200:
            # Create the folder if it does not exist
            os.makedirs(f'Trade Sales/{folder_name}', exist_ok=True)
            
            # Initialize the image counter for the folder if not already set
            if folder_name not in image_counters:
                image_counters[folder_name] = 1  # Start counting from 1
            
            # Get the current image counter for the folder
            img_counter = image_counters[folder_name]
            img_path = os.path.join('Trade Sales', folder_name, f'image_{img_counter}.jpg')
            
            with open(img_path, 'wb') as f:
                f.write(img_response.content)
            print(f'Downloaded {img_path} from {url}')
            # Increment the image counter for the folder
            image_counters[folder_name] += 1
        else:
            print(f'Failed to download image from {url}')
    except TypeError:
        if url[0] == 'O' or url[0] == 'W':
            print(f'{folder_name}: {url} is a server file path, please access manually')
        else:
            print(f'Likely URL gave 404. Please check {folder_name}: {url}')
        pass

# Initialize dictionary to store image counters per folder
image_counters = {}

# Loop through the DataFrame and download each image
for index, row in trade_promo_pivot.iterrows():
    folder_name = row['module']
    url = row['URL']
    download_image(url.strip(), folder_name.strip(), image_counters)

Downloaded Trade Sales\FOOD1\image_1.jpg from https://www.petbarn.com.au/savourlife-ancient-grains-gluten-free-lamb-rice-adult-dog-food?sku=138840
Downloaded Trade Sales\FOOD2\image_1.jpg from https://www.petbarn.com.au/advance-total-wellbeing-chicken-all-breed-adult-dog-food?sku=25721
Downloaded Trade Sales\FOOD3\image_1.jpg from https://www.petbarn.com.au/probalance-total-health-chicken-dog-food?sku=3218927
Downloaded Trade Sales\FOOD4\image_1.jpg from https://www.petbarn.com.au/pure-life-natural-boost-salmon-adult-dog-food?sku=140791
Downloaded Trade Sales\FOOD5\image_1.jpg from https://www.petbarn.com.au/black-hawk-original-chicken-adult-cat-food?sku=146670
Downloaded Trade Sales\FOOD6\image_1.jpg from https://www.petbarn.com.au/advance-indoor-chicken-rice-adult-cat-food?sku=136333
Downloaded Trade Sales\LITTER1\image_1.jpg from https://www.petbarn.com.au/breeder-s-choice-plus-probiotic-cat-litter?sku=146403
Downloaded Trade Sales\LITTER2\image_1.jpg from https://www.petbarn.com.au